In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import keras
import tensorflow as tf

In [ ]:
from keras import utils

In [ ]:
plant_train = utils.image_dataset_from_directory(
                '/content/drive/MyDrive/archive (6)/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train',
                label_mode = "int",
                labels="inferred",
                color_mode = "rgb",
                image_size = (200,200)

)

Found 70437 files belonging to 38 classes.


In [ ]:
plant_validation = utils.image_dataset_from_directory(
                '/content/drive/MyDrive/archive (6)/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid',
                label_mode = "int",
                labels="inferred",
                color_mode = "rgb",
                image_size = (200,200),
                subset = 'training',
                seed = 123,
                validation_split = 0.3

)

Found 17582 files belonging to 38 classes.
Using 12308 files for training.


In [ ]:
plant_test = utils.image_dataset_from_directory(
                '/content/drive/MyDrive/archive (6)/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid',
                label_mode = "int",
                labels="inferred",
                color_mode = "rgb",
                image_size = (200,200),
                subset = 'validation',
                seed = 123,
                validation_split = 0.3

)

Found 17582 files belonging to 38 classes.
Using 5274 files for validation.


In [ ]:
class_names = plant_train.class_names

In [ ]:
len(class_names)

38

In [ ]:
tf.data.experimental.cardinality(plant_train).numpy()

2202

In [ ]:
tf.data.experimental.cardinality(plant_validation).numpy()

385

In [ ]:
tf.data.experimental.cardinality(plant_test).numpy()

165

In [ ]:
autotune = tf.data.AUTOTUNE

In [ ]:
plant_train =  plant_train.prefetch(buffer_size=autotune)

In [ ]:
plant_test =  plant_test.prefetch(buffer_size=autotune)

In [ ]:
plant_validation =  plant_validation.prefetch(buffer_size=autotune)

In [ ]:
data_augmentation = keras.Sequential([
    keras.layers.RandomFlip('horizontal'),
    keras.layers.RandomRotation(0.2)
])

In [ ]:
preprocess_input = keras.applications.resnet50.preprocess_input

In [ ]:
base_model = keras.applications.ResNet50(include_top=False,
                                         input_shape=(200,200,3),
                                         weights = 'imagenet'

                                         )

94765736/94765736 [==============================] - 1s 0us/step


In [ ]:
base_model.trainable = False

In [ ]:
len(base_model.trainable_weights)

0

In [ ]:
len(base_model.non_trainable_weights)

318

In [ ]:
len(class_names)

38

In [ ]:
global_average_layer = keras.layers.GlobalAvgPool2D()

In [ ]:
inputs  = keras.layers.Input(shape=(200,200,3))
x = preprocess_input(inputs)
x = base_model(x, training=False)
x = global_average_layer(x)
prediction_layer = keras.layers.Dense(400,activation='relu',kernel_initializer='he_normal')(x)
prediction_layer = keras.layers.Dense(219,activation='relu',kernel_initializer='he_normal')(prediction_layer)
output = keras.layers.Dense(38,activation='softmax',kernel_initializer='he_normal')(prediction_layer)


In [ ]:
model = keras.models.Model(inputs = inputs,outputs = output)

In [ ]:
lr = 0.0001

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate = lr),loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),metrics=['accuracy'])

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 200, 200, 3)]     0         
                                                                 
 tf.__operators__.getitem (  (None, 200, 200, 3)       0         
 SlicingOpLambda)                                                
                                                                 
 tf.nn.bias_add (TFOpLambda  (None, 200, 200, 3)       0         
 )                                                               
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                             

In [ ]:
epoch = 20
callbacks = keras.callbacks.EarlyStopping(patience=3,monitor='val_loss')

In [ ]:
history = model.fit(plant_train,validation_data=plant_validation,epochs=1,callbacks=callbacks)

1565/2202 [====================>.........] - ETA: 1:19:19 - loss: 0.4820 - accuracy: 0.8848

In [ ]:
loss = history.history['loss']
val_loss =  history.history['val_loss']

In [ ]:
plt.plot(history.epoch,loss,label='Training loss')
plt.plot(history.epoch,val_loss,label='Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Plant Training Cross-Validation')

In [ ]:
model.compile(optimizer=keras.optimizer.RMSprop(learning_rate =lr/10),loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),metrics=['accuracy'])

In [ ]:
final_epoch = 40

In [ ]:
history_fine = model.fit(plant_train,validation_data=plant_validation,epochs=epoch,callbacks=callbacks,initial_epoch=history.epoch[-1],epochs=1)

In [ ]:
pred = model.predict(plant_test)

In [ ]:
plt.plot(history_fine.epoch,loss,label='Training loss')
plt.plot(history_fine.epoch,val_loss,label='Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Plant Training Cross-Validation')